<a href="https://colab.research.google.com/github/nforesperance/TensorFlow/blob/master/retrain_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

!pip install pyyaml h5py  # Required to save models in HDF5 format

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense

print(tf.version.VERSION)

2.1.0


In [0]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = (train_images.reshape((train_images.shape[0], 28, 28, 1)))/ 255.0
test_images = (test_images.reshape((test_images.shape[0], 28, 28, 1))) / 255.0
test_images = test_images / 255.0
train_images = train_images / 255.0

In [0]:
# Define a simple sequential model
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
  model.add(MaxPool2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model
# Create a basic model instance
model = create_model()

In [0]:
checkpoint_path = "trained0/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
save_path = "trained0"

#N:B The Followind Code is ran only once

# Create a callback that saves the model's weights
easystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, mode='min', baseline=None, restore_best_weights=True)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                            monitor='val_loss',
                            verbose=1,
                            save_weights_only=True,
                            save_best_only=True,
                            mode='min')

# Train the model with the new callback
model.fit(train_images, train_labels, validation_data=(train_images,
                                                       train_labels), 
                                                       epochs=20, 
                                                       batch_size=50, 
                                                       verbose = 2,
                                                       callbacks=[checkpoint,easystopping])
# model.load_weights(checkpoint_path)
# model.save("trained1/best.h5")

In [0]:
model.save(save_path+"/best.h5")

In [61]:
for i in range (10):

  easystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, mode='min', baseline=None, restore_best_weights=True)

  checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                            monitor='val_loss',
                            verbose=1,
                            save_weights_only=True,
                            save_best_only=True,
                            mode='min')

  # Train the model with the new callback
  model.fit(train_images, train_labels, validation_data=(train_images,
                                                       train_labels), 
                                                       epochs=20, 
                                                       batch_size=50, 
                                                       verbose = 2,
                                                       callbacks=[checkpoint,easystopping])
  model.save(save_path+"/best.h5")


Train on 60000 samples, validate on 60000 samples
Epoch 1/20

Epoch 00001: val_loss improved from inf to 1.46880, saving model to trained0/cp.ckpt
60000/60000 - 9s - loss: 1.4688 - accuracy: 0.9931 - val_loss: 1.4688 - val_accuracy: 0.9933
Epoch 2/20

Epoch 00002: val_loss improved from 1.46880 to 1.46777, saving model to trained0/cp.ckpt
60000/60000 - 9s - loss: 1.4687 - accuracy: 0.9933 - val_loss: 1.4678 - val_accuracy: 0.9938
Epoch 3/20

Epoch 00003: val_loss did not improve from 1.46777
Restoring model weights from the end of the best epoch.
60000/60000 - 9s - loss: 1.4688 - accuracy: 0.9930 - val_loss: 1.4682 - val_accuracy: 0.9938
Epoch 00003: early stopping
Train on 60000 samples, validate on 60000 samples
Epoch 1/20

Epoch 00001: val_loss improved from inf to 1.46871, saving model to trained0/cp.ckpt
60000/60000 - 9s - loss: 1.4687 - accuracy: 0.9933 - val_loss: 1.4687 - val_accuracy: 0.9933
Epoch 2/20

Epoch 00002: val_loss improved from 1.46871 to 1.46800, saving model to tr

In [0]:
model.save(save_path+"/best.h5")

In [62]:
# model.load_weights(checkpoint_path)
# model.save("trained/best.h5")
del model
model = tf.keras.models.load_model(save_path+'/best.h5')
# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


10000/10000 - 1s - loss: 1.4791 - accuracy: 0.9827
Restored model, accuracy: 98.27%


In [63]:
# deeper cnn model for mnist
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPool2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(MaxPool2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# define model
		model = define_model()
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=2, batch_size=32, validation_data=(testX, testY), verbose=1)
		# evaluate model
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# stores scores
		scores.append(acc)
		histories.append(history)
	return scores, histories

# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		pyplot.subplot(2, 1, 1)
		pyplot.title('Cross Entropy Loss')
		pyplot.plot(histories[i].history['loss'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		pyplot.subplot(2, 1, 2)
		pyplot.title('Classification Accuracy')
		pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	pyplot.show()

# summarize model performance
def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	pyplot.boxplot(scores)
	pyplot.show()

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# evaluate model
	scores, histories = evaluate_model(trainX, trainY)
	# learning curves
	# summarize_diagnostics(histories)
	# summarize estimated performance
	# summarize_performance(scores)

# entry point, run the test harness
run_test_harness()

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 13s 262us/sample - loss: 0.1455 - accuracy: 0.9541 - val_loss: 0.0623 - val_accuracy: 0.9805
Epoch 2/2
48000/48000 [==============================] - 12s 250us/sample - loss: 0.0471 - accuracy: 0.9857 - val_loss: 0.0539 - val_accuracy: 0.9848
> 98.483
Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 12s 257us/sample - loss: 0.1422 - accuracy: 0.9559 - val_loss: 0.0648 - val_accuracy: 0.9822
Epoch 2/2
48000/48000 [==============================] - 12s 252us/sample - loss: 0.0476 - accuracy: 0.9851 - val_loss: 0.0469 - val_accuracy: 0.9866
> 98.658
Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 12s 259us/sample - loss: 0.1414 - accuracy: 0.9554 - val_loss: 0.0604 - val_accuracy: 0.9801
Epoch 2/2
48000/48000 [==============================] - 12s 250us/sample - loss: 